In [ ]:
import os
from jiwer import wer, cer, compute_measures
import pandas as pd

def lire_txt(fichier):
    with open(fichier, 'r', encoding='utf-8') as f:
        return f.read().strip()

def comparer_transcriptions(dossier_hypothese, dossier_reference):
    resultats = []

    fichiers_communs = set(os.listdir(dossier_hypothese)).intersection(os.listdir(dossier_reference))

    for nom_fichier in fichiers_communs:
        chemin_hyp = os.path.join(dossier_hypothese, nom_fichier)
        chemin_ref = os.path.join(dossier_reference, nom_fichier)

        ref = lire_txt(chemin_ref)
        hyp = lire_txt(chemin_hyp)

        mesures = compute_measures(ref, hyp)

        resultats.append({
            "Fichier": nom_fichier,
            "WER (%)": round(mesures["wer"] * 100, 2),
            "CER (%)": round(cer(ref, hyp) * 100, 2),
            "Insertions": mesures["insertions"],
            "Suppressions": mesures["deletions"],
            "Substitutions": mesures["substitutions"],
        })

    df_resultats = pd.DataFrame(resultats)
    moyenne = df_resultats.mean(numeric_only=True)
    moyenne["Fichier"] = "MOYENNE"
    df_resultats = pd.concat([df_resultats, pd.DataFrame([moyenne])], ignore_index=True)

    return df_resultats

# Dossiers
dossier_paul = r"C:\2. Stage\1. Corpus Ground truth\2. Comparaison transcription manuelle selon transcripteur\Paul"
dossier_clemence = r"C:\2. Stage\1. Corpus Ground truth\2. Comparaison transcription manuelle selon transcripteur\Clémence"
dossier_capucine = r"C:\2. Stage\1. Corpus Ground truth\2. Comparaison transcription manuelle selon transcripteur\Capucine"

# Comparaisons
df_clemence_vs_paul = comparer_transcriptions(dossier_clemence, dossier_paul)
df_capucine_vs_paul = comparer_transcriptions(dossier_capucine, dossier_paul)
df_clémence_vs_capucine = comparer_transcriptions(dossier_clemence, dossier_capucine)

# Feuille de synthèse
def extraire_moyenne(df, nom_comparaison):
    moyenne = df[df["Fichier"] == "MOYENNE"].copy()
    moyenne.insert(0, "Comparaison", nom_comparaison)
    return moyenne.drop(columns=["Fichier"])

df_synthese = pd.concat([
    extraire_moyenne(df_clemence_vs_paul, "Clémence vs Paul"),
    extraire_moyenne(df_capucine_vs_paul, "Capucine vs Paul"),
    extraire_moyenne(df_clémence_vs_capucine, "Clémence vs Capucine")
], ignore_index=True)

# Export dans un seul fichier Excel avec plusieurs feuilles
with pd.ExcelWriter("1. Comparaison_transcriptions_trois_transcripteurs.xlsx") as writer:
    df_clemence_vs_paul.to_excel(writer, sheet_name="Clémence_vs_Paul", index=False)
    df_capucine_vs_paul.to_excel(writer, sheet_name="Capucine_vs_Paul", index=False)
    df_clémence_vs_capucine.to_excel(writer, sheet_name="Clémence_vs_Capucine", index=False)
    df_synthese.to_excel(writer, sheet_name="Synthèse", index=False)
